Устанавливаем библиотеки: pandas - для работы с таблицами, pyreadstat - для чтения файлов из статистических программ (типа SPSS). Обязательно указываем версию библиотеки через ==, потому что код, работающий в одной версии, может не работать в другой.  
Документации на них:

https://pandas.pydata.org

https://ofajardo.github.io/pyreadstat_documentation/_build/html/index.html


In [1]:
!pip install pandas==1.4.2 pyreadstat==1.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Подключаем Google Disk, надо будет дать разрешение этой jupyter-notebook тетрадке на чтение/запись с него. Файлы *.sav должны быть заранее загружены на Google Disk. Если они загружены не в корень диска, а в подпапку, надо будет поменять переменные fpath (cм. ниже)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# импортируем библиотеку pyreadstat, 
# которая ориентирована на работу с форматами статистических (в т.ч., платных) программ.
import pyreadstat
import numpy as np

In [4]:
fpath_2014 = 'drive/MyDrive/коуж/IND_2014.sav'
df_2014, meta_2014 = pyreadstat.read_file_multiprocessing(pyreadstat.read_sav, fpath_2014, num_processes=16, encoding='cp1251') 

In [5]:
fpath_2016 = 'drive/MyDrive/коуж/IND_2016.sav'
df_2016, meta_2016 = pyreadstat.read_file_multiprocessing(pyreadstat.read_sav, fpath_2016, num_processes=16, encoding='cp1251') 

In [6]:
fpath_2018 = 'drive/MyDrive/коуж/IND_2018.sav'
df_2018, meta_2018 = pyreadstat.read_file_multiprocessing(pyreadstat.read_sav, fpath_2018, num_processes=16, encoding='cp1251') 

In [7]:
# В программе учебной дисциплины заявлен анализ по 2011 году, но в данных по нему нет нужных нам переменных.

# Источник данных: https://gks.ru/ .

fpath_2020 = 'drive/MyDrive/коуж/IND_OSN_2020.sav'
df_2020, meta_2020 = pyreadstat.read_file_multiprocessing(pyreadstat.read_sav, fpath_2020, num_processes=16, encoding='cp1251') 
selection = [
    'I08_05_01', # Цели выхода в сеть Интернет: выполнение оплачиваемой работы' 
    'I07_10_02_01', # 'Имеется доступ к сети Интернет',
    'I05_08', # 'Режим работы',
    'DEN_NA_DUSHU', # 'Денежный доход в среднем на члена ДХ, в месяц, рублей'
    "H00_02", # код субъекта РФ
    ] 

In [8]:
# часть нужных данных есть только для домохозяйств, а не для индивидуальных респондентов
fpath_household_2020 = 'drive/MyDrive/коуж/HHOLD_OSN_2020.sav'
df_household_2020, meta_household_2020 = pyreadstat.read_file_multiprocessing(pyreadstat.read_sav, fpath_household_2020, num_processes=16, encoding='cp1251') 

In [9]:
selection_household = [
    'H02_353', # 'Намерения в ближайшие 2-3 года изменить жилищные условия',
    'H02_355_01_01', #'Предполагают приобрести квартиру в многоквартирном строящемся доме',
    'H02_355_02_01', #'Предполагают приобрести квартиру на вторичном рынке',
    'H02_355_03_01', #'Предполагают приобрести жилой дом (часть дома)',
    'H02_355_04_01', #'Предполагают приобрести комнату / несколько комнат в квартире',
    'H02_355_05_01', #'Предполагают приобрести другое место для постоянного проживания',
    'H02_355_01_02', #'Предполагают приобрести квартиру в многоквартирном строящемся доме (общая площадь)',
    'H02_355_02_02', #'Предполагают приобрести квартиру на вторичном рынке (общая площадь)',
    'H02_355_03_02', #'Предполагают приобрести жилой дом (часть дома) (общая площадь)',
    'H02_355_04_02', #'Предполагают приобрести комнату / несколько комнат в квартире (общая площадь)',
    'H02_355_05_02', #'Предполагают приобрести другое место для постоянного проживания (общая площадь)',
    "H00_02", # код субъекта РФ
    ]

In [10]:
def descriptional_statistics(df, meta, selection):
    # Создадим новую таблицу (pandas DataFrame), но только с теми колонками,
    # которые используются в расчётах.
    # Так используется меньше памяти, удобнее визуализировать. Но можно пользоваться и исходной таблицей.
    selected = df[selection]
    # Используем генератор списков, чтобы переименовать служебные заголвки в читаемые.
    # для этого мы хотим "пройтись" по всем меткам в списке selection и получить новый список вместо selection.
    # Подробнее о генераторах списков: https://pythontutor.ru/lessons/lists/#section_3
    selection_names = [meta.column_names_to_labels[label] for label in selection]
    # Заменим исходные наименования колонок на наименования из получившегося списка selection_names, 
    # теперь у нас наименования колонок будет как в столбце "метки" файла sav.
    selected.columns = selection_names
    # 98 - код Республики Саха (Якутии), согласно ОКВЭДам.
    # создаем маску, чтобы получить таблицу с данными только по Якутии.
    mask = selected['Код субъекта РФ'] == '98'
    # применяем маску - используем только те строчки, где значения маски True
    region = selected[mask]
    # число наблюдений в таблице получаем через атрибут shape.
    # так как в shape находится объект типа кортеж (tuple)- структура данных, похожая на список,
    # но неизменяемая (чтобы добавить или удалить элемент, придется создавать новый объект типа кортеж в памяти).
    # из кортежа,  так же, как из списка, можно получить по порядковому индексу его элемент
    # в данном случае у нас shape[0] - количество строк, shape[1] - количество столбцов.
    # pandas DataFrame - двумерная таблица, ее размерность описывается количеством строчек и столбцов.
    # одномерная "таблица" pandas, по сути, одна колонка, размерность которой описывается только количеством строчек,
    # называется Series

    print(f"Количество наблюдений по выбранным переменным: {region.shape[0]}")
    # apply - метод pandas DataFrame, который позволяет применить функцию ко всему Series или DataFrame,
    # не прописывая цикл явно.

    # Применим функцию, который переведет служебные значения в читаемые ко всему столбцу Размер населенного пункта.

    # в python есть два способа объявить функцию - через ключевое слово def или lambda.
    # def удобно использовать, когда у функции много аргументов и/или в ней выполняется много действий
    # lambda удобно использовать, когда функция принимает 1-2 аргумента и выполняемые действия можно прописать в 
    # одну строчку

    # После ключевого слова lambda идет аргумент функции (x), двоеточие и после него прописываем действие 
    # над аргументом х

    # В данном случае необходимо из словаря variable_to_label получить значение ключа,
    # а затем по этому значению получить другое значение 
    # из словаря value_labels
    # т.е., value_labels это вложенный словарь - словарь в словаре.
    for name, variable in zip(selection_names[:-1], selection[:-1]):
        print(name)
        if (variable != 'DEN_NA_DUSHU') and not ('(общая площадь)' in name): # доходы и вопросы с общей площадью не по номинальной шкале, так что их преобразовывать не надо
            

            possible_answers = meta.value_labels[meta.variable_to_label[variable]]
            possible_answers_str = {}
            for k, v in possible_answers.items():
                possible_answers_str[str(k)] = v # бывает так, что в этом словаре часть ключей имеет тип данных float, а часть строка
            
            print(f"Возможные ответы: {possible_answers_str}")
            try: 
                variable_values = region[name].apply(lambda x: possible_answers_str[str(x)] if not np.isnan(x) else x)
            except KeyError:
                print("Варианты ответа отсутствуют в словаре возможных ответов. Посчитать осмысленные описательные статистики нельзя")
                print(f"Варианты ответа: {region[name].unique()}")
                print("--------------------------")
                continue
            print(variable_values.describe())
        else:
            print(region[name].describe())
        print("--------------------------")

Что считает метод describe для нечисловых данных:

unique - количество уникальных значений

top - наиболее часто встречающееся значение

freq - частота top-а (мода)

в случае нескольких одинаковых top-ов, какой вывести, выбирается случайно

Что посчитает метод describe для числовых данных:

count - количество наблюдений

mean - среднее арифметическое

std - среднеквадратическое отклонение, standard deviation

min - минимальное значение

max - максимальное значение

25%, 50%, 75% - Q1, Q2, Q3

In [11]:
print("2014 ГОД")
descriptional_statistics(df_2014, meta_2014, selection)

2014 ГОД
Количество наблюдений по выбранным переменным: 978
Цели выхода в сеть Интернет: поиск или выполнение оплачиваемой работы
Возможные ответы: {'1.0': 'Поиск или выполнения оплачиваемой работы, рассылка информаци'}
count                                                    61
unique                                                    1
top       Поиск или выполнения оплачиваемой работы, расс...
freq                                                     61
Name: Цели выхода в сеть Интернет: поиск или выполнение оплачиваемой работы, dtype: object
--------------------------
Имеется доступ к сети Интернет
Возможные ответы: {'1.0': 'Да', '2.0': 'Нет'}
count     36
unique     2
top       Да
freq      27
Name: Имеется доступ к сети Интернет, dtype: object
--------------------------
Режим работы
Возможные ответы: {'1.0': 'с полным рабочим днем (сменой) или полной рабочей неделей', '2.0': 'с неполным рабочим днем (сменой) или неполной рабочей неделе', '3.0': 'с гибким режимом работы (гибкий, ск

In [12]:
print("2016 ГОД")
descriptional_statistics(df_2016, meta_2016, selection)

2016 ГОД
Количество наблюдений по выбранным переменным: 1189
Цели выхода в сеть Интернет: поиск или выполнение оплачиваемой работы
Возможные ответы: {'1.0': 'Поиск или выполнения оплачиваемой работы, рассылка информаци'}
count                                                    47
unique                                                    1
top       Поиск или выполнения оплачиваемой работы, расс...
freq                                                     47
Name: Цели выхода в сеть Интернет: поиск или выполнение оплачиваемой работы, dtype: object
--------------------------
Имеется доступ к сети Интернет
Возможные ответы: {'1.0': 'Да', '2.0': 'Нет'}
count     39
unique     2
top       Да
freq      36
Name: Имеется доступ к сети Интернет, dtype: object
--------------------------
Режим работы
Возможные ответы: {'1.0': 'С полным рабочим днем (сменой) или полной рабочей неделей', '2.0': 'С неполным рабочим днем (сменой) или неполной рабочей неделе', '3.0': 'С гибким режимом работы (гибкий, с

In [13]:
print("2018 ГОД")
descriptional_statistics(df_2018, meta_2018, selection)

2018 ГОД
Количество наблюдений по выбранным переменным: 1158
Цели выхода в сеть Интернет: выполнение оплачиваемой работы
Возможные ответы: {'1.0': 'Цели выхода в сеть Интернет: выполнение оплачиваемой работы'}
count                                                    43
unique                                                    1
top       Цели выхода в сеть Интернет: выполнение оплачи...
freq                                                     43
Name: Цели выхода в сеть Интернет: выполнение оплачиваемой работы, dtype: object
--------------------------
Имеется доступ к сети Интернет
Возможные ответы: {'1.0': 'Да', '2.0': 'Нет'}
count     40
unique     1
top       Да
freq      40
Name: Имеется доступ к сети Интернет, dtype: object
--------------------------
Режим работы
Возможные ответы: {'1.0': 'С полным рабочим днем (сменой) или полной рабочей неделей', '2.0': 'С неполным рабочим днем (сменой) или неполной рабочей неделе', '3.0': 'С гибким режимом работы (гибкий, скользящий рабочий ден

In [14]:
print("2020 ГОД")
descriptional_statistics(df_2020, meta_2020, selection)


2020 ГОД
Количество наблюдений по выбранным переменным: 1178
Цели выхода в сеть Интернет: выполнение оплачиваемой работы
Возможные ответы: {'1.0': 'Цели выхода в сеть Интернет: выполнение оплачиваемой работы'}
count                                                   118
unique                                                    1
top       Цели выхода в сеть Интернет: выполнение оплачи...
freq                                                    118
Name: Цели выхода в сеть Интернет: выполнение оплачиваемой работы, dtype: object
--------------------------
Имеется доступ к сети Интернет
Возможные ответы: {'1.0': 'Да', '2.0': 'Нет'}
count     51
unique     2
top       Да
freq      46
Name: Имеется доступ к сети Интернет, dtype: object
--------------------------
Режим работы
Возможные ответы: {'1.0': 'С полным рабочим днем (сменой) или полной рабочей неделей', '2.0': 'С неполным рабочим днем (сменой) или неполной рабочей неделе', '3.0': 'С гибким режимом работы (гибкий, скользящий рабочий ден

In [15]:
# в программе учебной дисциплины заявлено использование данных за 2011, 2014 и 2016 годы
# но в них отсутствуют необходимые переменные, поэтому рассматриваем только 2018 и 2020

fpath_household_2018 = 'drive/MyDrive/коуж/HHOLD_2018.sav'
df_household_2018, meta_household_2018 = pyreadstat.read_file_multiprocessing(pyreadstat.read_sav, fpath_household_2018, num_processes=16, encoding='cp1251') 


In [16]:
print("2018 ГОД")
descriptional_statistics(df_household_2018, meta_household_2018, selection_household)
# примечание: вариант -7 в вопросах про общую площадь это "затрудняюсь ответить"

2018 ГОД
Количество наблюдений по выбранным переменным: 486
Намерения в ближайшие 2-3 года изменить жилищные условия
Возможные ответы: {'-7.0': 'Затруднились ответить', '1.0': 'Да', '2.0': 'Нет'}
count     486
unique      3
top       Нет
freq      323
Name: Намерения в ближайшие 2-3 года изменить жилищные условия, dtype: object
--------------------------
Предполагают приобрести квартиру в многоквартирном строящемся доме
Возможные ответы: {'1.0': 'Да', '2.0': 'Нет'}
count      54
unique      2
top       Нет
freq       36
Name: Предполагают приобрести квартиру в многоквартирном строящемся доме, dtype: object
--------------------------
Предполагают приобрести квартиру на вторичном рынке
Возможные ответы: {'1.0': 'Да', '2.0': 'Нет'}
count      54
unique      2
top       Нет
freq       40
Name: Предполагают приобрести квартиру на вторичном рынке, dtype: object
--------------------------
Предполагают приобрести жилой дом (часть дома)
Возможные ответы: {'1.0': 'Да', '2.0': 'Нет'}
count      5

In [17]:
print("2020 ГОД")
descriptional_statistics(df_household_2020, meta_household_2020, selection_household)

2020 ГОД
Количество наблюдений по выбранным переменным: 486
Намерения в ближайшие 2-3 года изменить жилищные условия
Возможные ответы: {'-7.0': 'Затруднились ответить', '1.0': 'Да', '2.0': 'Нет'}
count     486
unique      3
top       Нет
freq      343
Name: Намерения в ближайшие 2-3 года изменить жилищные условия, dtype: object
--------------------------
Предполагают приобрести квартиру в многоквартирном строящемся доме
Возможные ответы: {'1.0': 'Да', '2.0': 'Нет'}
count      33
unique      2
top       Нет
freq       26
Name: Предполагают приобрести квартиру в многоквартирном строящемся доме, dtype: object
--------------------------
Предполагают приобрести квартиру на вторичном рынке
Возможные ответы: {'1.0': 'Да', '2.0': 'Нет'}
count      33
unique      2
top       Нет
freq       18
Name: Предполагают приобрести квартиру на вторичном рынке, dtype: object
--------------------------
Предполагают приобрести жилой дом (часть дома)
Возможные ответы: {'1.0': 'Да', '2.0': 'Нет'}
count      3